# Fragestellung 1.3: -> Export nur Energiequellen GAS - HEIZÖL - ELEKTRIZITÄT
## Welche Gebiete in einer Gemeinde sind am stärksten betroffen?

Um diese Frage zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. <br>
[Link zum GWR](https://www.housing-stat.ch/de/index.html)<br>
[Link zur GWR Dokumentation](https://www.housing-stat.ch/de/docs/index.html)<br>
[Link zum GWR Download](https://www.housing-stat.ch/de/madd/public.html)<br>
<br>
Die Daten werden nach Gemeinde und Energiequelle gruppiert und dann die einzelnen Objekte gezählt. <br>
Anschliessend wird die resultierende Liste als Geopackage zur weiterverarbeitung bzw. Visualisierung exportiert.<br>
<br>
---
<i> CAS Spatial Data Analytics 2022 </i> ¦ <i> Kommunale Übersicht von Heizsystemen und Energieträgern in Wohngebäuden </i> ¦ <i> Stand: 22.09.2022  </i> ¦ <i> Entwickler: Jürg Reist </i>

### Notebook vorbereiten und die benötigten Daten aus dem GWR einlesen

In [1]:
import pandas as pd
import geopandas as gpd

#lesen GWR Daten
dfGWRSource = pd.read_csv('GWR_Data/gebaeude_batiment_edificio_CH.csv', usecols=['GGDENAME','GKODE', 'GKODN', 'GENH1'],  sep='\t')


In [2]:
#Column mit geeigneten Namen
dfGWRSource = dfGWRSource.rename(columns={"GGDENAME": "Gemeinde",
                                          "GKODE": "lat",
                                          "GKODN": "lon",
                                          "GENH1":"Energiequelle"
                                          })

In [3]:
dfGWRSource

,Gemeinde,lat,lon,Energiequelle
0,Aeugst am Albis,2679647.268,1237500.347,7530.0
1,Aeugst am Albis,2680635.900,1236936.229,7530.0
2,Aeugst am Albis,2679016.828,1235828.156,7530.0
3,Aeugst am Albis,2679677.869,1237453.256,7530.0
4,Aeugst am Albis,2679692.997,1237491.284,7530.0
...,...,...,...,...
3053218,La Baroche,2582768.637,1251359.261,7530.0
3053219,La Baroche,2582743.072,1251406.135,7530.0
3053220,La Baroche,2584600.501,1251495.804,7520.0
3053221,La Baroche,2583393.948,1251448.443,7540.0


In [4]:
# Die korrekten Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
dfGWRSource['Energiequelle'] = dfGWRSource.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Luft',
                                            7510: 'Erdwärme',
                                            7511: 'Erdwärme',
                                            7512: 'Erdwärme',
                                            7513: 'Wasser',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Holz',
                                            7541: 'Holz',
                                            7542: 'Holz',
                                            7543: 'Holz',
                                            7550: 'Abwärme',
                                            7560: 'Elektrizität',
                                            7570: 'Sonne',
                                            7580: 'Fernwärme',
                                            7581: 'Ferwärme',
                                            7582: 'Fernwärme',
                                            7598: 'Unbestimmt',
                                            7599: 'Keine'
                                            })

In [5]:
dfGWRSource

,Gemeinde,lat,lon,Energiequelle
0,Aeugst am Albis,2679647.268,1237500.347,Heizöl
1,Aeugst am Albis,2680635.900,1236936.229,Heizöl
2,Aeugst am Albis,2679016.828,1235828.156,Heizöl
3,Aeugst am Albis,2679677.869,1237453.256,Heizöl
4,Aeugst am Albis,2679692.997,1237491.284,Heizöl
...,...,...,...,...
3053218,La Baroche,2582768.637,1251359.261,Heizöl
3053219,La Baroche,2582743.072,1251406.135,Heizöl
3053220,La Baroche,2584600.501,1251495.804,Gas
3053221,La Baroche,2583393.948,1251448.443,Holz


In [6]:
#nur die fossilen Energiequellen filtern
options = ['Gas', 'Heizöl', 'Elektrizität']
dfGWRfossileEnergiequelle = dfGWRSource[(dfGWRSource['Energiequelle'].isin(options))] 

dfGWRfossileEnergiequelle

                                                 

,Gemeinde,lat,lon,Energiequelle
0,Aeugst am Albis,2679647.268,1237500.347,Heizöl
1,Aeugst am Albis,2680635.900,1236936.229,Heizöl
2,Aeugst am Albis,2679016.828,1235828.156,Heizöl
3,Aeugst am Albis,2679677.869,1237453.256,Heizöl
4,Aeugst am Albis,2679692.997,1237491.284,Heizöl
...,...,...,...,...
3053211,La Baroche,2583088.040,1251351.424,Heizöl
3053217,La Baroche,2583355.208,1251442.906,Heizöl
3053218,La Baroche,2582768.637,1251359.261,Heizöl
3053219,La Baroche,2582743.072,1251406.135,Heizöl


In [7]:
#Nullwerte anschauen
dfGWRfossileEnergiequelle_null = dfGWRfossileEnergiequelle.isnull().sum()
dfGWRfossileEnergiequelle_null

Gemeinde         0
lat              7
lon              7
Energiequelle    0
dtype: int64

### Daten als Geopackage zur Weiterverarbeitung bzw. Visusliserung exportieren

In [8]:
gdf = gpd.GeoDataFrame(
    dfGWRfossileEnergiequelle, geometry=gpd.points_from_xy(dfGWRfossileEnergiequelle.lat, dfGWRfossileEnergiequelle.lon)
)

In [9]:
gdf

,Gemeinde,lat,lon,Energiequelle,geometry
0,Aeugst am Albis,2679647.268,1237500.347,Heizöl,POINT (2679647.268 1237500.347)
1,Aeugst am Albis,2680635.900,1236936.229,Heizöl,POINT (2680635.900 1236936.229)
2,Aeugst am Albis,2679016.828,1235828.156,Heizöl,POINT (2679016.828 1235828.156)
3,Aeugst am Albis,2679677.869,1237453.256,Heizöl,POINT (2679677.869 1237453.256)
4,Aeugst am Albis,2679692.997,1237491.284,Heizöl,POINT (2679692.997 1237491.284)
...,...,...,...,...,...
3053211,La Baroche,2583088.040,1251351.424,Heizöl,POINT (2583088.040 1251351.424)
3053217,La Baroche,2583355.208,1251442.906,Heizöl,POINT (2583355.208 1251442.906)
3053218,La Baroche,2582768.637,1251359.261,Heizöl,POINT (2582768.637 1251359.261)
3053219,La Baroche,2582743.072,1251406.135,Heizöl,POINT (2582743.072 1251406.135)


In [10]:
#Datenframe für Verteilung der Energietrager pro Gemeinde
dfEnergyHeatmap = gdf[['Gemeinde','Energiequelle', 'geometry']]
dfEnergyHeatmap

,Gemeinde,Energiequelle,geometry
0,Aeugst am Albis,Heizöl,POINT (2679647.268 1237500.347)
1,Aeugst am Albis,Heizöl,POINT (2680635.900 1236936.229)
2,Aeugst am Albis,Heizöl,POINT (2679016.828 1235828.156)
3,Aeugst am Albis,Heizöl,POINT (2679677.869 1237453.256)
4,Aeugst am Albis,Heizöl,POINT (2679692.997 1237491.284)
...,...,...,...
3053211,La Baroche,Heizöl,POINT (2583088.040 1251351.424)
3053217,La Baroche,Heizöl,POINT (2583355.208 1251442.906)
3053218,La Baroche,Heizöl,POINT (2582768.637 1251359.261)
3053219,La Baroche,Heizöl,POINT (2582743.072 1251406.135)


### Daten in ein GeoPackage-File exportieren

In [11]:
#Als GeoPackage-File exportieren
gdf.to_file('Export_Data/GeoPackage/EnergiequelleFossile.gpkg', driver='GPKG', limit=0)


ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
